In [16]:
%pwd

'd:\\Projects\\ML Projects\\End-to-End Wine Quality\\End-to-End-ML-Project'

In [3]:
import os
os.chdir('../')
%pwd

'd:\\Projects\\ML Projects\\End-to-End Wine Quality\\End-to-End-ML-Project'

In [25]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str

In [26]:
from MLProject.constants import *
from MLProject.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.RandomForestClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name
           
        )

        return model_evaluation_config

In [ ]:
import pandas as pd
import os
from pathlib import Path
from MLProject import logger
from MLProject.utils.common import save_json
from sklearn.metrics import classification_report, confusion_matrix
import joblib
import numpy as np

# Function to convert numpy types inside a dictionary
def convert_numpy_in_dict(obj):
    if isinstance(obj, dict):
        return {k: convert_numpy_in_dict(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_numpy_in_dict(item) for item in obj]
    elif isinstance(obj, (np.integer, np.int64)):
        return int(obj)
    elif isinstance(obj, (np.floating, np.float64)):
        return float(obj)
    elif isinstance(obj, np.bool_):
        return bool(obj)
    else:
        return obj

# Model Evaluation Class
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def eval_model(self):
        model = joblib.load(self.config.model_path)
        test_data = pd.read_csv(self.config.test_data_path)
        
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        pred_y = model.predict(test_x)

        # Get structured classification report
        report = classification_report(test_y, pred_y, output_dict=True)

        # Get confusion matrix
        matrix = confusion_matrix(test_y, pred_y)

        scores = {
            "classification_report": report,
            "confusion_matrix": {
                "labels": list(model.classes_),
                "matrix": matrix.tolist()
            }
        }

        # Convert numpy types BEFORE saving
        clean_scores = convert_numpy_in_dict(scores)

        save_json(path=Path(self.config.metric_file_name), data=clean_scores)


In [37]:

try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.eval_model()
except Exception as e:
    raise e

[2025-04-27 13:21:37,069: INFO: common: yaml file: config\config.yaml loaded successfully.]
[2025-04-27 13:21:37,071: INFO: common: yaml file: params.yaml loaded successfully.]
[2025-04-27 13:21:37,079: INFO: common: yaml file: schema.yaml loaded successfully.]
[2025-04-27 13:21:37,079: INFO: common: Created directory at: artifacts]
[2025-04-27 13:21:37,079: INFO: common: Created directory at: artifacts/model_evaluation]
[2025-04-27 13:21:37,377: INFO: common: Json file saved at: artifacts\model_evaluation\metrics.json]
